- Exercici 1

Realitza web scraping d'una pàgina de la borsa de Madrid (https://www.bolsamadrid.es) utilitzant BeautifulSoup i Selenium.

S'utilitza selenium per accedir a l'apartat d'accions de tecnologia i telecomunicacions.

In [19]:
from selenium import webdriver
from bs4 import BeautifulSoup
from selenium.webdriver.support.ui import Select
import requests
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import re


driver = webdriver.Firefox()
driver.get("https://www.bolsamadrid.es")
assert "Bolsa de Madrid" in driver.title
acciones = driver.find_element_by_link_text('Acciones').click()
menu = Select(driver.find_element_by_id('SelSector'))
menu.select_by_visible_text('Tecnología y Telecomunicaciones')
consultar = driver.find_element_by_id('ctl00_Contenido_Consultar').click()

URL = driver.current_url
page = requests.get(URL)

In [2]:
soup = BeautifulSoup(page.content, "html.parser")
print(soup.prettify())

<!DOCTYPE html PUBLIC "-//W3C//DTD XHTML 1.0 Transitional//EN" "http://www.w3.org/TR/xhtml1/DTD/xhtml1-transitional.dtd">
<html xmlns="http://www.w3.org/1999/xhtml">
 <head data-analytics-id="UA-35966870-2" data-app-path="/" data-bolsa="BMadrid" data-hora-act="Wed, 29 Sep 2021 18:52:08 GMT" data-idioma="esp">
  <meta content="IE=11" http-equiv="X-UA-Compatible"/>
  <meta content="text/html; charset=utf-8" http-equiv="Content-Type"/>
  <meta content="Copyright © BME 2021" id="ctl00_copyright" name="copyright"/>
  <title>
   Bolsa de Madrid - Precios de la sesión
  </title>
  <link href="/esp/aspx/RSS/RSS.ashx?feed=Todo" id="ctl00_RSSLink1" rel="alternate" title="Bolsa de Madrid: Todos los contenidos agregados" type="application/rss+xml"/>
  <link href="/esp/aspx/RSS/RSS.ashx?feed=NotasPrensa" id="ctl00_RSSLink2" rel="alternate" title="Bolsa de Madrid: Notas de Prensa" type="application/rss+xml"/>
  <link href="/esp/aspx/RSS/RSS.ashx?feed=Regulacion" id="ctl00_RSSLink3" rel="alternate" t

Utilitzem BeautifulSoup per scrapejar les accions. Posteriorment les guardarem en un DataFrame que guardarem en un archiu csv

In [42]:
tabla = soup.find(id="ctl00_Contenido_tblAcciones")

rows = tabla.find_all('tr')
i=0
columns = []
header = []
for row in rows:
    if i==0:
        cols = row.find_all('th')
        cols=[x.text.strip() for x in cols]
        header.extend(cols)
    else:
        cols=row.find_all('td')
        cols=[x.text.strip() for x in cols]
        columns.append(cols)
    i+=1

print(header)
print(columns)

['Nombre', 'Últ.', '% Dif.', 'Máx.', 'Mín.', 'Volumen', 'Efectivo (miles €)', 'Fecha', 'Hora']
[['ACCIONA', '144,5000', '0,84', '147,1000', '143,9000', '94.724', '13.746,16', '29/09/2021', 'Cierre'], ['ACERINOX', '11,2150', '4,67', '11,3050', '10,7400', '1.328.029', '14.808,05', '29/09/2021', 'Cierre'], ['ACS', '23,6200', '0,55', '23,7800', '23,4000', '872.640', '20.625,37', '29/09/2021', 'Cierre'], ['AENA', '148,9000', '3,08', '149,1500', '144,0000', '217.565', '31.996,03', '29/09/2021', 'Cierre'], ['ALMIRALL', '13,5800', '-0,95', '13,7600', '13,1200', '404.503', '5.482,80', '29/09/2021', 'Cierre'], ['AMADEUS', '57,6400', '1,26', '57,7600', '56,8400', '519.455', '29.879,51', '29/09/2021', 'Cierre'], ['ARCELORMIT.', '25,8800', '1,85', '26,2500', '25,3450', '431.217', '11.162,69', '29/09/2021', 'Cierre'], ['B.SANTANDER', '3,1565', '2,48', '3,1630', '3,0990', '50.825.844', '159.495,46', '29/09/2021', 'Cierre'], ['BA.SABADELL', '0,7266', '3,65', '0,7330', '0,7054', '51.684.271', '37.208,1

In [43]:
ibex = pd.DataFrame(data=columns, columns=header)
ibex

,Nombre,Últ.,% Dif.,Máx.,Mín.,Volumen,Efectivo (miles €),Fecha,Hora
0,ACCIONA,"144,5000","0,84","147,1000","143,9000",94.724,"13.746,16",29/09/2021,Cierre
1,ACERINOX,"11,2150","4,67","11,3050","10,7400",1.328.029,"14.808,05",29/09/2021,Cierre
2,ACS,"23,6200","0,55","23,7800","23,4000",872.640,"20.625,37",29/09/2021,Cierre
3,AENA,"148,9000","3,08","149,1500","144,0000",217.565,"31.996,03",29/09/2021,Cierre
4,ALMIRALL,"13,5800","-0,95","13,7600","13,1200",404.503,"5.482,80",29/09/2021,Cierre
5,AMADEUS,"57,6400","1,26","57,7600","56,8400",519.455,"29.879,51",29/09/2021,Cierre
6,ARCELORMIT.,"25,8800","1,85","26,2500","25,3450",431.217,"11.162,69",29/09/2021,Cierre
7,B.SANTANDER,"3,1565","2,48","3,1630","3,0990",50.825.844,"159.495,46",29/09/2021,Cierre
8,BA.SABADELL,"0,7266","3,65","0,7330","0,7054",51.684.271,"37.208,14",29/09/2021,Cierre
9,BANKINTER,"5,0620","1,63","5,1100","5,0140",2.013.132,"10.187,64",29/09/2021,Cierre


In [44]:
ibex.columns

Index(['Nombre', 'Últ.', '% Dif.', 'Máx.', 'Mín.', 'Volumen',
       'Efectivo (miles €)', 'Fecha', 'Hora'],
      dtype='object')

eliminar los puntos y cambiar las comas por puntos

In [48]:
nums = ['Últ.','% Dif.','Máx.','Mín.','Volumen','Efectivo (miles €)']

for num in nums:
    #ibex[num] = ibex[num].replace(',', '.')
    ibex[num].astype('float64')
    #pd.to_numeric(ibex[num])

ValueError: could not convert string to float: '144,5000'

In [47]:
ibex

,Nombre,Últ.,% Dif.,Máx.,Mín.,Volumen,Efectivo (miles €),Fecha,Hora
0,ACCIONA,"144,5000","0,84","147,1000","143,9000",94.724,"13.746,16",29/09/2021,Cierre
1,ACERINOX,"11,2150","4,67","11,3050","10,7400",1.328.029,"14.808,05",29/09/2021,Cierre
2,ACS,"23,6200","0,55","23,7800","23,4000",872.640,"20.625,37",29/09/2021,Cierre
3,AENA,"148,9000","3,08","149,1500","144,0000",217.565,"31.996,03",29/09/2021,Cierre
4,ALMIRALL,"13,5800","-0,95","13,7600","13,1200",404.503,"5.482,80",29/09/2021,Cierre
5,AMADEUS,"57,6400","1,26","57,7600","56,8400",519.455,"29.879,51",29/09/2021,Cierre
6,ARCELORMIT.,"25,8800","1,85","26,2500","25,3450",431.217,"11.162,69",29/09/2021,Cierre
7,B.SANTANDER,"3,1565","2,48","3,1630","3,0990",50.825.844,"159.495,46",29/09/2021,Cierre
8,BA.SABADELL,"0,7266","3,65","0,7330","0,7054",51.684.271,"37.208,14",29/09/2021,Cierre
9,BANKINTER,"5,0620","1,63","5,1100","5,0140",2.013.132,"10.187,64",29/09/2021,Cierre


In [15]:

sns.boxplot(data=ibex, x='Nombre', y=['Últ.','% Dif.','Máx.','Mín.','Volumen','Efectivo (miles €)'])

ValueError: No variables found for grid columns.

In [5]:
driver.close()
ibex.to_csv('ibex_35_tecno_teleco.csv')